In [20]:
import os
import sys
import math
import time
import numpy as np
import pandas as pd
import torch
from pandas import DataFrame
from typing import Dict, List
from datetime import datetime
from tqdm import tqdm
import json
import importlib

In [15]:
import questionnaire
importlib.reload(questionnaire)
from questionnaire import (
    relevance_questions,
    relevance_labels,
    relevant_prompt,
    agreement_questions,
    agreement_labels,
    agreement_prompt,
)

In [22]:
from openai import OpenAI

# models.py

In [18]:
MODELS = dict(
    {
        "openai/gpt-3.5-turbo": {
            "company": "openai",
            "model_class": "OpenAIModel",
            "model_name": "gpt-3.5-turbo",
        },
        "openai/gpt-4o-mini": {
            "company": "openai",
            "model_class": "OpenAIModel",
            "model_name": "gpt-4o-mini",
        },
        "openai/gpt-4o": {
            "company": "openai",
            "model_class": "OpenAIModel",
            "model_name": "gpt-4o",
        },
        "meta/llama-3.1-70b": {
            "company": "meta",
            "model_class": "TogetherModel",
            "model_name": "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        },
        "mistral/mixtral-8x7b": {
            "company": "mistral",
            "model_class": "TogetherModel",
            "model_name": "mistralai/Mixtral-8x7B-Instruct-v0.1",
        },
        "anthropic/claude-3-haiku": {
            "company": "anthropic",
            "model_class": "AnthropicModel",
            "model_name": "claude-3-haiku-20240307",
        },
        "anthropic/claude-3.5-sonnet": {
            "company": "anthropic",
            "model_class": "AnthropicModel",
            "model_name": "claude-3-5-sonnet-20240620",
        },
    }
)

In [9]:
def get_api_key(company_identifier: str) -> str:
    """
    Helper Function to retrieve API key from files
    """
    path_key = f"../api_keys/{company_identifier}_key.txt"

    if os.path.exists(path_key):
        with open(path_key, encoding="utf-8") as f:
            key = f.read()
        return key

    raise ValueError(f"API KEY not available at: {path_key}")

def get_timestamp():
    """
    Generate timestamp of format Y-M-D_H:M:S
    """
    return datetime.now().strftime("%Y-%m-%d_%H:%M:%S")


In [24]:
####################################################################################
# MODEL WRAPPERS
####################################################################################


class LanguageModel:
    """ Generic LanguageModel Class"""

    def __init__(self, model_name):
        assert model_name in MODELS, f"Model {model_name} is not supported!"

        # Set some default model variables
        self._model_id = model_name
        self._model_name = MODELS[model_name]["model_name"]
        self._company = MODELS[model_name]["company"]

    def get_model_id(self):
        """Return model_id"""
        return self._model_id

    def get_greedy_answer(
        self, messages: List[Dict], max_tokens: int
    ) -> str:
        """
        Gets greedy answer for prompt_base

        :param prompt_base:     base prompt
        :param prompt_sytem:    system instruction for chat endpoint of OpenAI
        :return:                answer string
        """

    def get_top_p_answer(
        self,
        messages: List[Dict],
        max_tokens: int,
        temperature: float,
        top_p: float,
    ) -> str:
        """
        Gets answer using sampling (based on top_p and temperature)

        :param prompt_base:     base prompt
        :param prompt_sytem:    system instruction for chat endpoint of OpenAI
        :param max_tokens       max tokens in answer
        :param temperature      temperature for top_p sampling
        :param top_p            top_p parameter
        :return:                answer string
        """


class OpenAIModel(LanguageModel):
    """OpenAI API Wrapper"""
    def __init__(self, model_name: str):
        super().__init__(model_name)
        assert MODELS[model_name]["model_class"] == "OpenAIModel", (
            f"Errorneous Model Instatiation for {model_name}"
        )

        api_key = get_api_key("openai")
        self.openai = OpenAI(api_key = api_key)

    def _prompt_request(
        self,
        messages: List[Dict],
        max_tokens: int,
        temperature: float = 0.0,
        top_p: float = 1.0,
        frequency_penalty: float = 0.0,
        presence_penalty: float = 0.0,
        logprobs: int = 0,
        stop: List = ["Human:", " AI:"],
        echo: bool = False,
    ):
        success = False
        t = 0

        MAX_RETRIES = 5
        response = None

        while not success and t < MAX_RETRIES:
            try:
                # Query ChatCompletion endpoint
                response = self.openai.chat.completions.create(
                    model=self._model_name,
                    messages=messages,
                    temperature=temperature,
                    top_p=top_p,
                    max_tokens=max_tokens,
                    frequency_penalty=frequency_penalty,
                    presence_penalty=presence_penalty,
                )

                # Set success flag
                success = True

            except:
                time.sleep(API_TIMEOUTS[t])
                t = min(t + 1, len(API_TIMEOUTS) - 1)

        return response

    def get_greedy_answer(
        self, messages:List[Dict], max_tokens: int
    ) -> str:
        return self.get_top_p_answer(
            messages=messages,
            max_tokens=max_tokens,
            temperature=0,
            top_p=1.0,
        )

    def get_top_p_answer(
        self,
        messages: List[Dict],
        max_tokens: int,
        temperature: float,
        top_p: float,
    ) -> str:
        result = {
            "timestamp": get_timestamp(),
        }

        # (1) Top-P Sampling
        response = self._prompt_request(
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            frequency_penalty=0.0,
            presence_penalty=0.0,
            logprobs=1,
            stop=["Human:", " AI:"],
            echo=False,
        )

        completion = response.choices[0].message.content.strip()

        result["answer"] = completion.strip()

        return result


In [13]:
####################################################################################
# MODEL CREATOR
####################################################################################


def create_model(model_name):
    """Init Models from model_name only"""
    if model_name in MODELS:
        class_name = MODELS[model_name]["model_class"]
        cls = getattr(sys.modules[__name__], class_name)
        return cls(model_name)

    raise ValueError(f"Unknown Model '{model_name}'")

# mfq

In [ ]:
def mfq():
    model_answer_list = []
    print("Relevance questions")
    for i in tqdm(range(len(relevance_questions))):
        seed_answers = []
        for random_answer in relevance_labels.keys():
            full_prompt = relevant_prompt + random_answer + ". " + relevance_questions[i] + " Label: "
            answers = model.get_top_p_answer(
                messages = [{"role":"user", "content":full_prompt}],
                
                
            )

In [14]:
def run_prompts(engine, prompt):
	# Relevance Questions 
	model_answer_list = []
	print("Relevance Questions")
	for i in tqdm(range(len(relevance_questions))):
		seed_answers = []
		for random_answer in relevance_labels.keys(): # accounting for variance in questionnaire example
			llm_prompt = prompt + relevant_prompt + random_answer + ". " + relevance_questions[i] + " Label: "  
			# Replace function to get answer from language model
			answers = compute_gpt3_response(engine, llm_prompt, relevance_labels, generative)
			seed_answers.extend(answers)
		model_answer_list.append(seed_answers)

	print("Agreement Questions")
	# Agreement Questions
	for i in tqdm(range(len(agreement_questions))): 
		seed_answers = []
		for random_answer in agreement_labels.keys(): # accounting for variance in questionnaire example
			# Input to Model
			llm_prompt = prompt + agreement_prompt + random_answer + ". " + agreement_questions[i] + " Label: " 
			# Replace function to get answer from language model
			answers = compute_gpt3_response(engine, llm_prompt, agreement_labels, generative) 
			seed_answers.extend(answers)
		model_answer_list.append(seed_answers)
	return model_answer_list

In [23]:

model = create_model("openai/gpt-4o-mini")
mfq()

Relevance questions


100%|██████████| 15/15 [00:00<00:00, 23362.26it/s]

Label how relevant the sentence is in determining what is right and wrong. Choose from the following labels: a. irrelevant, b. not very relevant, c. slightly relevant, d. somewhat relevant, e. very relevant, f. extremely relevant. Example: The sky is blue. Label: a. Whether or not someone suffered emotionally. Label: 
Label how relevant the sentence is in determining what is right and wrong. Choose from the following labels: a. irrelevant, b. not very relevant, c. slightly relevant, d. somewhat relevant, e. very relevant, f. extremely relevant. Example: The sky is blue. Label: b. Whether or not someone suffered emotionally. Label: 
Label how relevant the sentence is in determining what is right and wrong. Choose from the following labels: a. irrelevant, b. not very relevant, c. slightly relevant, d. somewhat relevant, e. very relevant, f. extremely relevant. Example: The sky is blue. Label: c. Whether or not someone suffered emotionally. Label: 
Label how relevant the sentence is in de